In [1]:
import importlib

import functions.data_preprocess as dp
importlib.reload(dp);

import functions.plot as dplt
importlib.reload(dplt);

import functions.transform_data as td
importlib.reload(td);

In [2]:
from dash import html, dcc, Dash, dash_table
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

import numpy as np
import pandas as pd
import plotly.express as px
# import plotly.io as pio
import plotly.graph_objects as go

# Dashboard

In [3]:
app = Dash(external_stylesheets=['assets/style_sheet.css', dbc.themes.BOOTSTRAP], title="Wars Consequences")

## Import & Preprocess Data

In [4]:
plots_theme = 'simple_white'
idp_color = '#9B1D20'
ref_color = '#235755'
asylm_color = '#E19A00'
gray = '#b3b3b3'
default_year = 2022

In [5]:
df_all_years, second_plot, lons_lats = dp.prepare_data();
td.get_df(df_all_years)

e:\ITI_AI\Phase 2\14.Data Viisualization\refugees_project\functions\data_preprocess.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df_all_years['Country of origin'] = df_all_years['Country of origin'].apply(lambda x: x.split('(')[0]).apply(lambda x: x.split(':')[0]).str.replace('Dem. Rep. of the', "").str.replace('United Rep. of', '').str.strip()


In [6]:
countries_second = list(second_plot["country"].unique())

In [7]:
total_three = df_all_years.groupby('Year').agg({'Asylum-seekers': 'sum', "Refugees under UNHCR's mandate": 'sum', "IDPs of concern to UNHCR": 'sum'})

In [8]:
iso_loc = lons_lats[['Alpha-3 code', 'Latitude (average)', 'Longitude (average)', 'Country']]
isos_original = df_all_years[['Country of origin (ISO)', 'Country of asylum (ISO)']]
dplt.get_isos(iso_loc, isos_original)

## Title & Description

In [9]:
title = html.Div(
    [
        html.Div(
            [
                html.Img(src=app.get_asset_url('people.png'),id="title_icon"),
                html.Span("Wars Consequences", id='title')
            ], className="mt-3")
    ], style={'margin': '0px 30px 10px'}
)

In [10]:
intro = html.Div([
    html.P(["The pain of war can't exceed the woe of aftermath."], className='h5'),
    html.P(["Every single day, families around the world are being forced from their homes. In search of safety and a better life, they are risking everything to escape conflict, disaster, poverty, or hunger, often leaving with nothing but the clothes they are wearing."]),
    ], style={'margin': '0px 730px 0px 30px'})

In [11]:
description = dbc.Row([
    html.H5("Population Types"),
    dbc.Col([
        html.H6(['Refugees '], style={'color':ref_color}),
        html.Span([" are people who have fled their homes, crossed an international border, and cannot return because they fear their lives are in danger. People become refugees when one or more of their basic human rights are violated or threatened."]),
    ], className='col-4 desc'),
    dbc.Col([
        html.H6(["IDPs"], style={'color':idp_color}),
        html.Span([" have been forced to flee their homes for the same reasons as refugees but have not crossed an international border. IDPs live in their countries of origin but are often afraid or unwilling to return home for fear of persecution."]),
    ], className='col-4 desc'),
    dbc.Col([
        html.H6(["Asylum-seekers"], style={'color':asylm_color}),
        html.Span([" are people who have fled their homes and claim international protection, but whose status has not yet been definitively determined. Every refugee begins as an asylum-seeker, but not every asylum-seeker will be granted refugee status."])
    ], className='col-4 desc')
    ], id="description_row")

## Analysis Tab

### Select Year

In [12]:
base_year = 2001
slider_marks = {str(i): str(i) for i in df_all_years[df_all_years['Year']>=base_year]['Year'].unique()}
max_year = df_all_years['Year'].max()

In [13]:
slider = dbc.Row([
       html.H5("Select A Year", className="me-1"),
       html.Br(),
        dcc.Slider(
            base_year,
            max_year,
            id='main_slider_id',
            step=None,
            value=2022,
            marks=slider_marks), html.Br()
        ], id='slider_row')


### BANS

In [14]:
ban1, ban2, ban3, ban4, ban5 = dplt.get_all_bans(default_year)

In [15]:
bans = dbc.Row([
    ban1,
    ban2, 
    ban3,
    ban4,
    ban5
],className='ban_row', id='bans1')

In [16]:
@app.callback(
Output(component_id = "bans1" , component_property = "children"),
Input(component_id = "main_slider_id" , component_property = "value"))
def update_bans(year):
    ban1, ban2, ban3, ban4, ban5 = dplt.get_all_bans(year)
    return [ban1, ban2, ban3, ban4, ban5]

### Populations By Country of Origin and Population Type, 2022

In [17]:
first_plot = td.get_first_data(default_year)

In [18]:
default_fig1 = dplt.plot_fig1(first_plot)

In [19]:
plot_1 = dbc.Col([
    dbc.Row([html.H5('Populations By Country of Origin and Population Type')]),
    dbc.Row([dcc.Graph(id="figure1", figure=default_fig1)])
])

In [20]:
# FIRST GRAPH 
@app.callback(
Output(component_id = "figure1" , component_property = "figure"),
Input(component_id = "main_slider_id" , component_property = "value"))
def update_My_GRAPH(year):
    df_new= td.get_first_data(year)
    fig = dplt.plot_fig1(df_new)

    return fig

### Events Description

In [21]:
default_event = second_plot[(second_plot["country"]=='Syria') & (second_plot["date"]=='2022-01')][["day", "event"]]

In [22]:
table_events = dplt.create_events_table(default_event, '2022-01')

In [23]:
plot_2 = dbc.Col([
    dbc.Row([html.H5("Incidents Affecting The Protection of IDPs And Refugees In 2022")]),
    dbc.Row([
        dbc.Col(html.Div([dcc.Dropdown(countries_second,id='demo-dropdown_country', value='Syria')]),width = 6),
        dbc.Col(html.Div([dcc.Dropdown(id='demo-dropdown_date', value='2022-01')]),width = 6),
    ]),
    dbc.Row([table_events], id="events")
])

In [24]:
@app.callback(
Output("events" , 'children'),
Input("demo-dropdown_date" , "value"),
State("demo-dropdown_country" , "value"))
def update_event(date,country):
    out = second_plot[(second_plot["country"]==country) & (second_plot["date"]==date)][["day", "event"]]
    table_events = dplt.create_events_table(out, date)
    return table_events

In [25]:
@app.callback(
Output(component_id = "demo-dropdown_date" , component_property = 'options'),    
Input(component_id = "demo-dropdown_country" , component_property = "value"))
def update_date(country):
    options = list(second_plot[second_plot["country"]==country]["date"].unique())
    return options

### Asylum-seekers, Refugees, IDPs along the years

In [26]:
three_lines_plot = px.line(data_frame=total_three, y=['Asylum-seekers', "Refugees under UNHCR's mandate", "IDPs of concern to UNHCR"], color_discrete_sequence=[asylm_color, ref_color, idp_color], template=plots_theme)
three_lines_plot.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor="right",
    x=1
), legend_title="Population Type");

In [27]:
plot_3 = dbc.Col([
    dbc.Row([html.H5('Trend Over Time By Population Type')]),
    dbc.Row([dcc.Graph(id='line_plot', figure=three_lines_plot)])
])

### IDPs

In [28]:
idps = td.get_idps(default_year)
default_idps= dplt.plot_idp(idps)

In [29]:
plot_4 = dbc.Col([
    dbc.Row([html.H5('Top Countries With IDPs')]),
    dbc.Row([dcc.Graph(id='idp_plot', figure=default_idps)])
], className='col-5 p-0')

In [30]:
@app.callback(
Output(component_id = "idp_plot" , component_property = "figure"),
Input(component_id = "main_slider_id" , component_property = "value"))
def update_idp(year):
    new_idps = td.get_idps(year)
    fig= dplt.plot_idp(new_idps)

    return fig

### Migrations Connections

In [31]:
origin_countries = df_all_years[['Country of origin', 'Country of origin (ISO)']].drop_duplicates().sort_values('Country of origin')
i = origin_countries[origin_countries['Country of origin'] == 'Stateless'].index[0]
origin_countries = origin_countries.drop(i)

In [32]:
origin_countries = origin_countries.set_index('Country of origin (ISO)')
options_df = origin_countries.to_dict()

In [33]:
default_fig = dplt.create_map()

In [34]:
plot_5 = dbc.Col([
    dbc.Row([html.H5("Refugees' Destination")]),
    dbc.Row([dcc.Dropdown(options_df['Country of origin'], value='SYR', id='source_country')]),
    dbc.Row([dcc.Graph(id='mig_plot', figure=default_fig)]),
], className='col-5 p-0')

In [35]:
@app.callback(
Output(component_id = "mig_plot" , component_property = "figure"),
Input(component_id = "source_country" , component_property = "value"))
def update_map(country_iso):
    new_fig = dplt.create_map(country_iso)
    return new_fig

### Country of Asylm Map

In [36]:
default_fig2 = dplt.plot_asylm_ref(td.get_ref_data(default_year))

In [37]:
plot_6 = dbc.Col([
    dbc.Row([html.H5("Refugees In Countries of Asylm")]),
    dbc.Row([dcc.Graph(id='asylm_plot', figure=default_fig2)]),
])

In [38]:
@app.callback(
Output(component_id = "asylm_plot" , component_property = "figure"),
Input(component_id = "main_slider_id" , component_property = "value"))
def update_map2(year):
    new_fig = dplt.plot_asylm_ref(td.get_ref_data(year))
    return new_fig

### Asylm Seekers

In [39]:
default_seekers = dplt.plot_seekers(td.get_seekers())

In [40]:
plot_7 = dbc.Col([
    dbc.Row([html.H5('Top Countries With Asylum Seekers')]),
    dbc.Row([dcc.Graph(id='seekers_plot', figure=default_seekers)])
], className='col-6 p-0')

In [41]:
@app.callback(
Output(component_id = "seekers_plot" , component_property = "figure"),
Input(component_id = "main_slider_id" , component_property = "value"))
def update_seeker(year):
    new_seekers = td.get_seekers(year)
    fig= dplt.plot_seekers(new_seekers)

    return fig

### Last Row in Analysis

In [42]:
first_row = dbc.Row([
    plot_3,
    plot_4
])

second_row = dbc.Row([
    plot_1
], style={'margin-top': '25px'})

third_row = dbc.Row([
    plot_5, plot_6
], style={'margin-top': '25px'})

fourth_row = dbc.Row([
    plot_7, plot_2
], style={'margin-top': '25px'})


## Donation Tab

### IDPs Video

In [43]:
donation_vid = dbc.Row([
    dbc.Col([ 
        html.H3("Who are Internally Displaced People?", style={"font-family": "serif", "text-align": "center"})
    ]),
    dbc.Col([ 
        html.Iframe(src='https://www.youtube.com/embed/DCzpVQkencw', height=315, width=690)
    ])
], style={'align-items': 'center'})

### Charities

In [44]:
## describtion of ecah charity 
uncif_dis =  """
UNICEF works for the rights of every child, every day, across the globe.
UNICEF was established in the aftermath of World War II to help children whose
lives and futures were at risk – no matter what country they were from. """

unchr_dis = ''' UNHCR, the UN Refugee Agency, is a global organization dedicated to saving lives, 
protecting rights and building a better future for refugees, forcibly displaced communities and stateless people.'''

In [45]:
ch_title = html.Div([   
            html.Span('Help ', className='h2', style={"font-family": "serif", "text-align": "center"}),
            html.Span('Syrians ', className='h1', style={"font-family": "serif", "text-align": "center", 'color': '#C10000'}),
            html.Span('!', className='h2', style={"font-family": "serif", "text-align": "center"})
], style={'text-align': 'center'})

In [46]:
charities = dbc.Row([
        dbc.Col([html.Div([
                    # player.DashPlayer(
                    #     id='video-player',
                    #     url='https://www.youtube.com/watch?v=8G4hUALJ9HQ&feature=youtu.be',
                    #     controls=True,
                    #      ) ,
            html.Iframe(id='video-player', src="https://www.youtube.com/embed/8G4hUALJ9HQ", height=382,width=700)
        ])],width =6),

        # charities contact 
        dbc.Col([
            # first one 
            html.Div(
                html.Img(src=r'https://th.bing.com/th/id/OIP.byDeKXkU-ruBpsbNCQXxpgHaEK?pid=ImgDet&rs=1', alt='unicef logo',width="100", style={'margin-bottom':'5px'}),
            ),

            html.P(uncif_dis),        
            html.A(
                    style = {"font-size": "larger",
                            "font-weight": "bold"},
                    href= "https://help.unicef.org/?country=EG&gclsrc=aw.ds&msclkid=3cd8317e04441b2395735557bfd8d36a&utm_source=bing&utm_medium=cpc&utm_campaign=DLV_EG_Search_Brand_Generic_EN&utm_term=unicef&utm_content=Brand_Exact",
                    children = ["Donate HERE "]
            ),        
            
            html.Br(),
            html.Br(),
            html.Hr(),

            # second one 
            html.Div(
                html.Img(src=r'https://th.bing.com/th/id/R.f47476cd2c53ddafff9f56b738fd6b93?rik=otNP6cPlRdJ%2bRA&pid=ImgRaw&r=0', alt='unhcr logo',width="68" , style={'margin-bottom':'5px'}),
            ),
            html.P(unchr_dis),
                         
            html.A(
                style = {"font-size": "larger",
                        "font-weight": "bold"},
                href= "https://www.unhcr.org/syria-emergency.html",
                children = ["Donate HERE "])        
                            
            ], style = {"text-align": "center"})
        ], style = {"align-items": "center"})


## Tabs

In [47]:
analysis_content = dbc.Card(
    dbc.CardBody(
        [
            dbc.Container([
                slider,
                bans,
                first_row,
                second_row,
                third_row,
                fourth_row
            ])
        ]
    ),
)

donation_content = dbc.Card(
    dbc.CardBody(
        [
            donation_vid,
            html.Hr(),
            ch_title,
            html.Br(),
            charities
        ],
    style={"background-color": 'white'}),
)

tabs = dbc.Tabs(
    [
        dbc.Tab(analysis_content, label="Analysis"),
        dbc.Tab(donation_content, label="Donation")
    ]
)

In [48]:
app.layout = html.Div(children=[
    title,
    # intro,
    html.Div(tabs, style={'margin': '0px 30px'}),
    description,
])

# Run Server

In [49]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
